# A Todo app in 5 minutes

In [ ]:
# Pkg.clone("https://github.com/essenciary/SearchLight.jl") ### Only supports v0.7 and up

In [ ]:
using Dates

mutable struct Todo
    subject::String
    completed::Bool
    due_at::Date
end

And add nice rendering.

In [ ]:
import Base.show

function show(io::IO, t::Todo)
  time_diff = t.due_at - Dates.today()
  time_diff = time_diff < Dates.Day(0) ? -time_diff : time_diff

  Base.printstyled(
    io,
    (t.completed ? "✅" : "❌") * " " * 
    t.subject *
    (t.completed ? " " : (t.due_at < Dates.today() ? " (due $time_diff ago)" : " due in $time_diff")),
    color = (t.due_at < Dates.today() ? :red : :green)
  )
end

In [ ]:
Todo("Hello JuliaCon", false, Dates.today())

In [ ]:
Todo("This is important!", false, Dates.today() + Dates.Day(3))

In [ ]:
Todo("This is done", true, Dates.today())

In [ ]:
Todo("This is overdue!", false, Dates.today() - Dates.Day(3))

## Adding persistence

In [ ]:
exit() # restarting the kernel

To turn our object into a model we need to: 
* extend `AbstractModel`
* add an identity property (of type `DbId`)

In [ ]:
using Dates, SearchLight

mutable struct Todo <: AbstractModel
    subject::String
    completed::Bool
    due_at::Date
    
    id::DbId
end

Optional but very useful: a default keywords constructor

In [ ]:
Todo(; 
    subject = "Your todo here", 
    completed = false, 
    due_at = Dates.today() + Dates.Day(7)
) = Todo(subject, completed, due_at, DbId())

SearchLight employes the concept of `Migrations` to manage the database schema. 

_Schema migration refers to the management of incremental, reversible changes to relational database schemas._ (Wikipedia)

It uses Julia files and a simple DSL to write database independent migrations. These are placed under version control and need to be hosted on the local file system. 

In [ ]:
SearchLight.config.db_migrations_folder = "."

It's time to connect to the database.

In [ ]:
Dict(
    "host"     => "127.0.0.1",
    "username" => "adrian",
    "port"     => 5432, 
    "database" => "todoapp_dev",
    "adapter"  => "PostgreSQL") |> SearchLight.Database.connect!

SearchLight uses the database itself to keep track of the migrations. For a new database we need to "initialise" it (first time only). 

In [ ]:
SearchLight.db_init()

Now we can create a migration for our table. 

In [ ]:
SearchLight.table_name(Todo) |> SearchLight.Generator.new_table_migration

Edit the migration file. 

Notice: 
* the file name contains the generation timestamp to avoid name clashes with other developers
* simple DSL: reads like English
* Julia-like types (no fiddling with DB specific types)
* database independent
* includes features for creating and droping tables and adding and removing columns and indexes

In [ ]:
Migration.status()

In [ ]:
Migration.last_up()

In [ ]:
Migration.status()

Let's make it easy to create random `Todo`s

In [ ]:
using Faker

function random()
  Todo(subject = Faker.sentence(), completed = rand([true,false]), due_at = Dates.today() + Dates.Day(rand(-30:30)))
end

In [ ]:
todo = random()

We can now check if the `Todo`s are persisted or not.

In [ ]:
is_persisted(todo)

And we can persist them!

In [ ]:
save!(todo)

In [ ]:
is_persisted(todo)

In [ ]:
todo.subject = "Hello JuliaCon!"

In [ ]:
save!(todo)

Let's go crazy with getting things done!

In [ ]:
for i in 1:20 
    save(random())
end

Now we have data to play with.

In [ ]:
all(Todo)

In [ ]:
first(Todo)

In [ ]:
last(Todo)

In [ ]:
rand(Todo)

In [ ]:
count(Todo)

But the true power is in the Finders API.

In [ ]:
find_one(Todo, 10) # find Todo with id 10

In [ ]:
find_by(Todo, :completed, true) # find all completed Todos

In [ ]:
# Find a Todo with this subject -- if it doesn't exist, create it
# This one does exist
todo = find_one_by_or_create(Todo, :subject, "Hello JuliaCon!") 

In [ ]:
is_persisted(todo)

In [ ]:
# This one doesn't exist
todo = find_one_by_or_create(Todo, :subject, "Hello PyCon!")

In [ ]:
is_persisted(todo)

In [ ]:
save!(todo)

In [ ]:
is_persisted(todo) && delete(todo)

Helper methods especially useful when working with forms data. 

In [ ]:
todo = create_with(
           Todo, 
Dict(:subject => "Hello London", 
     :due_at => Dates.today() + Dates.Day(12)))

In [ ]:
update_with!(
todo, 
Dict(:subject => "Bye London", 
     :due_at => Dates.today() + Dates.Day(2)))

In [ ]:
update_with!!(todo, Dict(:subject => "Hello Barcelona", :due_at => Dates.today() + Dates.Day(4)))

### The QueryBuilder

In [ ]:
using SearchLight.QueryBuilder

In [ ]:
find(Todo, where("subject LIKE ?", "%Hello%") + where("id >= ?", 5))

In [ ]:
q = where("completed = ?", false) + 
    where("due_at BETWEEN ? AND ?", Dates.today() - Dates.Day(7), Dates.today()) + 
    order(:due_at, :desc) + 
    limit(10)

In [ ]:
find(Todo, q)

### We can progressively enhance our model to provide more features
* Validators
* Callbacks
* Auto serialisation 
* Predefined query scopes

In [ ]:
exit() # restart the Kernel

In [ ]:
using Dates, SearchLight, SearchLight.Validation

mutable struct Todo <: AbstractModel
    subject::String
    completed::Bool
    due_at::Date
    
    id::DbId
    
    # validators
    validator::ModelValidator
    
    # callbacks
    before_save::Function
    after_save::Function
    on_save::Function
    on_find::Function
    after_find::Function
    
    Todo(;
    subject = "",
    completed = false,
    due_at = Dates.today() + Dates.Week(1),
        
    id = DbId(),

    validator = ModelValidator([
      ValidationRule(:subject, not_empty),
      ValidationRule(:due_at, date_in_the_future)
    ]),

    before_save = (m::Todo) -> begin
      @info "Before save"
    end,
    after_save = (m::Todo) -> begin
      @info "After save"
    end,
    on_save = (m::Todo, field::Symbol, value::Any) -> begin
      @info "On save"
    end,
    on_find = (m::Todo, field::Symbol, value::Any) -> begin
      @info "On find"
    end,
    after_find = (m::Todo) -> begin
      @info "After find"
    end
        
  ) = new(subject, completed, due_at, id, 
          validator,
          before_save, after_save, on_save,
          on_find, after_find,
          )
end

Define the validation rules:

In [ ]:
function not_empty(field::Symbol, m::T)::ValidationResult where {T<:AbstractModel}
  isempty(getfield(m, field)) && return ValidationResult(invalid, :not_empty, "should not be empty")

  ValidationResult(valid)
end

function date_in_the_future(field::Symbol, m::T)::ValidationResult where {T<:AbstractModel}
  getfield(m, field) < Dates.today() && return ValidationResult(invalid, :future_date, "should be in the future")

  ValidationResult(valid)
end

In [ ]:
Dict(
    "host"     => "127.0.0.1",
    "username" => "adrian",
    "port"     => 5432, 
    "database" => "todoapp_dev",
    "adapter"  => "PostgreSQL") |> SearchLight.Database.connect!

In [ ]:
using SearchLight.QueryBuilder

In [ ]:
overdue = find(Todo, where("due_at < ?", Dates.today()) + limit(1)) |> first

Overdue Todos can no longer be added.

In [ ]:
save!(overdue)

### The lower level API (SQL strings, DataFrames and raw responses)

We can retrive results as `DataFrame`.

In [ ]:
df = SearchLight.DataFrame(Todo, 
                        where("completed = ?", false) + 
                        where("due_at < ?", Dates.today()) + 
                        order(:due_at) + 
                        limit(10) + 
                        offset(2))

In [ ]:
df = SearchLight.DataFrame(Todo, 
                            select(sql"COUNT(id)", :due_at) + 
                            where("completed = ?", false) + 
                            group(:due_at))

We can retrieve the autogenerated SQL as a `String`.

In [ ]:
todo = rand_one!!(Todo)

In [ ]:
query_sql = SearchLight.sql(Todo, where("due_at > ?", Dates.today()) + limit(1))

And we can use it to run a query. It will return a DataFrame. 

In [ ]:
SearchLight.query(query_sql)

We can also retrieve the raw result. 

In [ ]:
SearchLight.query_raw(query_sql)

In [ ]:
SearchLight.sql(todo)

In [ ]:
todo = random()

In [ ]:
SearchLight.sql(todo)